In [1]:
#Last updated: March 8, 2024

In [2]:
#Permanent cell 1

import os
import sys
import re
import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import subprocess
import shutil
import sqlite3
import mikeio1d
from mikeio1d.res1d import Res1D
import ctypes
MessageBox = ctypes.windll.user32.MessageBoxA

In [3]:
#Permanent cell 2

# user input
skip_first_day = True #Must match system assessment import

# #FSA vfd, 0
# model_area = 'FSA'
# map_folder=r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_0\SA_Maps_V187_Ann0p8_0p3cms"
# model_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_0\Model_V187_Ann0p8_0p3cms"
# model = "FSA_2050pop_V187_BSF_14k_Sealed_VFD.sqlite"
# result_lookup_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_0\HTML_Plots_V187_Ann0p8_0p3cms"
# ps_csv = 'PS_Capacity_FSA_2030_Network.csv'

# #FSA vfd, 1A
# model_area = 'FSA'
# map_folder=r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\Maps_V187_Ann0p8_UpsNSIRX1p2m(A)_0p3cms"
# model_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\Model_V187_Ann0p8_UpsNSIRX1p2m(A)_0p3cms"
# model = "FSA_2050pop_V187_BSF_14k_Sealed_VFD.sqlite"
# result_lookup_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\HTML_V187_Ann0p8_UpsNSIRX(A)_0p3cms"
# ps_csv = 'PS_Capacity_FSA_2030_Network.csv'


# #FSA vfd, 1B
# model_area = 'FSA'
# map_folder=r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\Maps_V187_Ann0p8_UpsNSIRX2m(B)_0p3cms"
# model_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\Model_V187_Ann0p8_UpsNSIRX2m(B)_0p3cms"
# model = "FSA_2050pop_V187_BSF_14k_Sealed_VFD.sqlite"
# result_lookup_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\HTML_V187_Ann0p8_UpsNSIRX2m(B)_0p3cms"
# ps_csv = 'PS_Capacity_FSA_2030_Network.csv'


#FSA vfd, 1C
model_area = 'FSA'
map_folder=r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\Maps_V187_Ann0p8_UpsNSR(C)_0p3cms"
model_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\Model_V187_Ann0p8_UpsNSR(C)_0p3cms"
model = "FSA_2050pop_V187_BSF_14k_Sealed_VFD.sqlite"
result_lookup_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\HTML_V187_Ann0p8_UpsNSR(C)_0p3cms"
ps_csv = 'PS_Capacity_FSA_2030_Network.csv'



In [4]:
#Permanent cell 3

try:

    # import model data
    # sql function
    def sql_to_df(sql,model):
      con = sqlite3.connect(model)
      df = pd.read_sql(sql, con)
      con.close()
      return df


    model_path = model_folder + '\\' + model

    if not (model.lower().endswith('.sqlite') or model.lower().endswith('.mdb')):
        raise ValueError("The variable 'model' must have .mdb or .sqlite extension: " + model)
    if os.path.exists(model_path) == False:
        raise ValueError("The variable 'model' points to a path that does not exist: " + model)    

    sql = "SELECT msm_Pump.AssetName, msm_Link.MUID "
    sql += "FROM (msm_Pump INNER JOIN msm_Node ON msm_Pump.fromnodeid = msm_Node.MUID) "
    sql += "INNER JOIN msm_Link ON msm_Node.MUID = msm_Link.tonodeid "
    sql += "WHERE msm_Pump.active = 1 AND msm_Pump.enabled = 1 AND msm_Link.active = 1 "
    sql += "GROUP BY msm_Pump.AssetName, msm_Node.MUID, msm_Link.MUID "

    pump_inflow_pipe_df = sql_to_df(sql,model_path)
    pump_inflow_pipe_df

    sql = "SELECT AssetName, MUID "
    sql += "FROM msm_Pump WHERE active = 1 AND msm_Pump.enabled = 1"

    pump_outflow_df = sql_to_df(sql,model_path)
    
except Exception as e:    
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 3\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")


In [5]:
#Permanent cell 4

# creating dictionaries

try:

    manual_adjusts = []
    manual_adjusts.append(['28k','28p0k'])
    manual_adjusts.append(['14k','14p0k'])

    #Import variable from System_Assessment_Consolidation_Variables.py
    sys.path.append(map_folder)
    from System_Assessment_Consolidation_Variables import consolidationRanks
    sys.path.remove(map_folder)
    ps_dict={}
    for consolidationRank in consolidationRanks:
        ps_dict[consolidationRank[0]] = consolidationRank[1].lower()

    color_dict={}
    color_dict['green'] = ['lightgreen','black']
    color_dict['blue'] = ['aqua','black']
    color_dict['rm-blue'] = ['lightgreen','black']
    color_dict['red'] = ['red','black']
    color_dict['rm-red'] = ['red','black']
    color_dict['purple'] = ['blueviolet','white']
    color_dict['yellow'] = ['gold','black']
    color_dict['orange'] = ['darkorange','black']
    color_dict['darkred'] = ['darkred','white']
    color_dict['black'] = ['black','white']

    color_dict_ps = {}
    color_dict_ps['green'] = ['lightgreen','black']
    color_dict_ps['red'] = ['darkred','white']
    color_dict_ps['black'] = ['black','white']

    consolidation_acronym = pd.read_csv(map_folder+"\\Consolidation_By_Acronym.csv")
    consolidation_acronym["Tab_ID"] = consolidation_acronym.acronym

    consolidation_by_pipe = pd.read_csv(map_folder+"\\Consolidation_By_Pipes.csv")
    consolidation_events = pd.read_csv(map_folder + "\\Consolidation_Events.csv",dtype={'simyear': float,'msm_node_muid': str,'msm_link_muid': str},low_memory=False)
    consolidation_events.dropna(subset=['consolidation_group'], inplace=True)
    consolidation_events['simyear'] = consolidation_events['simyear'].astype(int)
    consolidation_events["Pipe_Key"]= consolidation_events.consolidation_group.astype(str) + "-" + consolidation_events.msm_link_muid.astype(str) +"-" + consolidation_events.simyear.astype(str)
    consolidation_events.set_index(["Pipe_Key"],inplace=True)
    consolidation_ps = pd.read_csv(map_folder+"\\PS_Consolidation.csv").dropna()
    results = list(consolidation_events.result.unique())

    ps_stats = pd.read_csv(map_folder + "\\PS_Stats.csv")
    ps_stats = ps_stats[ps_stats['ResultFile'].isin(results)]

    # ps_stats['Consolidation_Group'] = ps_stats['ResultFile'].astype(str).str[:-45]
    if len(list(consolidation_by_pipe.consolidation_group.unique())) == 1:
        consolidation_ps['Consolidation_Group'] = consolidation_by_pipe.consolidation_group.unique()[0]
        ps_stats['Consolidation_Group'] = consolidation_by_pipe.consolidation_group.unique()[0]
    else:
        consolidation_ps['Consolidation_Group'] = consolidation_ps['map'].astype(str).apply(lambda x: re.search(r'(.*?\d+k)', x).group(1) if re.search(r'(.*?\d+k)', x) else None)
        consolidation_ps.dropna(inplace=True)    
        ps_stats['Consolidation_Group'] = ps_stats['ResultFile'].str.extract(r'^(.*?)(?=\d{4}p)')
        ps_stats['Consolidation_Group'] = ps_stats['Consolidation_Group'].str[:-1]
        for manual_adjust in manual_adjusts:
            consolidation_ps['Consolidation_Group'] = consolidation_ps['Consolidation_Group'].str.replace(manual_adjust[0], manual_adjust[1])
            ps_stats['Consolidation_Group'] = ps_stats['Consolidation_Group'].str.replace(manual_adjust[0], manual_adjust[1])


    ps_stats['Year'] = ps_stats['ResultFile'].str.extract(r'(\d{4})p')
    ps_stats['Year'] = ps_stats['Year'].astype(int)

    ps_results = ps_stats[['ResultFile','Year','Consolidation_Group']].groupby(['ResultFile','Year','Consolidation_Group']).sum()
    ps_results.reset_index(inplace=True)
    ps_results.set_index(['ResultFile'],inplace=True)
    
except Exception as e:    
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 4\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")


In [ ]:
#Permanent cell 5

# import results
try:

    first_flow = True
    for result in list(consolidation_events.result.unique()):
        result_found = False
        for f in os.listdir(model_folder):
            if f[-7:] == '.sqlite':
                result_subfolder = os.path.basename(f)[:-7] + '_m1d - Result Files'
                try:
                    for f1 in os.listdir(model_folder + '\\' + result_subfolder):
                        if f1.lower() == result.lower():
                            result_found = True
                            result_path = model_folder + '\\' + result_subfolder + '\\' + f1
                            print(result_path)
                except:
                    pass
        if not result_found:
            raise ValueError("Result file '" + result + "' not found.")

        print('Importing ' + result)
        res1d = Res1D(result_path)
        
        timestep_seconds = (max(res1d.time_index) - min(res1d.time_index)).total_seconds() / (len(res1d.time_index)-1) 
        skip_steps = int(86400 / timestep_seconds) if skip_first_day else 0

    #     for index, row in pump_inflow_pipe_df.iterrows():
        for index, row in pump_outflow_df.iterrows():
            muid = row['muid']
            ps = row['assetname']
            if ps in list(ps_stats.PS.unique()):
                values = list(res1d.query.GetReachEndValues('Pump:' + muid, 'Discharge'))[skip_steps:]
                flow_df = pd.DataFrame(index = res1d.time_index[skip_steps:])
                flow_df['PS'] = ps
                flow_df['Year'] = ps_results.loc[result,'Year']
                flow_df['Consolidation_Group'] = ps_results.loc[result,'Consolidation_Group']
                flow_df['MUID'] = muid
                flow_df['DateTimeRef'] = flow_df.index
                flow_df['Discharge'] = values

                if first_flow == True:
                    flow_df_all = flow_df.copy()
                else:
                    flow_df_all = pd.concat([flow_df_all,flow_df])
                first_flow = False

    flow_df_all

    flow_df_ps = flow_df_all.groupby(['PS', 'Year', 'Consolidation_Group', 'DateTimeRef']).sum()
    flow_df_ps.reset_index(inplace=True)
    flow_df_ps.Discharge = flow_df_ps.Discharge*1000
    time_bookends = [min(flow_df_ps.DateTimeRef),max(flow_df_ps.DateTimeRef)]

except Exception as e:    
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 5\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")




J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_1\Model_V187_Ann0p8_UpsNSR(C)_0p3cms\FSA_2030pop_V187_BSF_16p8k_Sealed_VFD_m1d - Result Files\FSA_BSF_16p8k_2030pop_S_V_2030_NetworkDefault_Network_HD.res1d
Importing FSA_BSF_16p8k_2030pop_S_V_2030_NetworkDefault_Network_HD.res1d


In [ ]:
#Permanent cell 6

try:
    #Make PS htmls
    years = list(consolidation_events.simyear.unique())
    max_sim_year = max(years)
    max_first_year = max(list(consolidation_acronym.firstyear.unique()))

    html_path = map_folder + '\\Acronym_Summary'
    try:
        os.makedirs(html_path)
    except:
        pass

    shutil.copy2('style.css', html_path + '\\style.css')
    shutil.copy2('script.js', html_path + '\\script.js')

    for consolidation_group in list(consolidation_ps.Consolidation_Group.unique()):
        print('Creating html for ' + consolidation_group)
        consolidation_group_df = consolidation_ps[consolidation_ps.Consolidation_Group == consolidation_group]

        f = open(html_path + '\\'+ consolidation_group + '_PS.html', "w")
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write('<script src="script.js"></script>\n')
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write('<!DOCTYPE html>\n')
        f.write('<html>\n')
        f.write('<head>\n')
        f.write('<meta charset="utf-8">\n')
        f.write('</head>\n')
        f.write('<body>\n\n')

        f.write('<div class="tab">\n')
        for index, row in consolidation_group_df.iterrows():
            tab = row["ps"] 
            first_year = row['firstyear']
            color = ps_dict[first_year]

            bg_color = color_dict[color][0]
            text_color = color_dict[color][1]

            f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  
                    + ')" style="background-color:' + bg_color + '; color:' + text_color + ';">' + tab + '</button>\n')
        f.write('</div>\n')

        for index, row in consolidation_group_df.iterrows():
            ps = row["ps"]
            tab = ps
            f.write('<div id="' + tab + '" class="tabcontent">\n')  
            f.write('<h1>' + ps + '</h1>')


            f.write('<div class="row"><div class="column">\n') 

    #         PS INDEX

            f.write('<table style=\'width: 60%;\'>\n')
            f.write('<tr>\n')
            f.write('<th>Legend</th>\n')
            f.write('</tr>\n')
            f.write('\n')
            f.write('<tr><td style="background-color: black; color:white">> Station capacity </td></tr>\n')
            f.write('<tr><td style="background-color: darkred; color:white">> Firm capacity </td></tr>\n')
            f.write('<tr><td style="background-color: lightgreen; color:black"> < Firm capacity </td></tr>\n')     
            f.write('</table>\n')
            f.write('<br>\n')



    #         PS table


            f.write('<table style=\'width: 50%;\'>\n')
            f.write('<tr>\n')
            f.write('<th rowspan="2">Year</th>\n') 

            f.write('<th>PS Outflow</th>\n') 
            f.write('<th>Firm Capacity</th>\n') 
            f.write('<th>Station Capacity</th>\n') 
            f.write('</tr>\n')
            f.write('<th colspan="3" style="text-align: center">Discharge (L/s)</th>\n')    
            f.write('<tr>\n')



            for i,year in enumerate(years):

                ps_stats_row = ps_stats[(ps_stats.Year == year)&(ps_stats.Consolidation_Group == consolidation_group)&(ps_stats.PS == ps)].copy()
                ps_stats_row.reset_index(inplace = True)
                discharge = round(ps_stats_row.loc[0,'MaxTotalOutflow'],1)
                firm_capacity = round(ps_stats_row.loc[0,'FirmCapacity'],0)
                station_capacity = round(ps_stats_row.loc[0,'StationCapacity'],0)
                color = ps_stats_row.loc[0,'Color'].lower()

                if color == 'red':
                    color = 'darkred'

                bg_color = color_dict[color][0]
                text_color = color_dict[color][1]


                f.write('<tr>\n')



                f.write('<td>'+ str(year) + '</td>\n')

                f.write('<td style="background-color:'+ bg_color + '; color:' + text_color + ';">' + str(discharge)+'</td>\n')
                if i == 0:
                    f.write('<td rowspan="' + str(len(years)) + '" style="text-align: center">' + str(firm_capacity) + '</td>\n')    
                    f.write('<td rowspan="' + str(len(years)) + '"style="text-align: center">' + str(station_capacity) + '</td>\n')

                f.write('</tr>\n')
            f.write('</table>\n')


            f.write('</div>\n')             
            f.write('<div class="column">\n')

            fig = go.Figure()

            firm_capacity = ps_stats[ps_stats.PS==ps].iloc[0,4]
            station_capacity = ps_stats[ps_stats.PS==ps].iloc[0,5]

            fig.add_trace(go.Scatter(x=time_bookends, 
                                     y = [station_capacity,station_capacity], 
                                     mode='lines',line_color='black',name='Station Capacity')) 

            fig.add_trace(go.Scatter(x=time_bookends, 
                                     y = [firm_capacity,firm_capacity], 
                                     mode='lines',line_color='red',name='Firm Capacity'))   

            for year in years:
                flow_df_ps_single = flow_df_ps[(flow_df_ps.PS==ps) & (flow_df_ps.Consolidation_Group==consolidation_group)
                                              & (flow_df_ps.Year==year)]
                fig.add_trace(go.Scatter(x = flow_df_ps_single.DateTimeRef, 
                                         y = flow_df_ps_single.Discharge,name=str(year)))
            fig.update_layout(
            title='Inflow to ' + ps,
            autosize=False,
            width = 1000,
            height=600,
            margin=dict(
                l=50,
                r=50,
                b=50,
                t=50,
                pad=4
                ),
                yaxis_title = 'Discharge (L/s)'
            )


            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))      

            f.write('</div>\n')
            f.write('</div>\n')  
            f.write("</div>\n")

        f.write('</body>\n')
        f.write('</html>\n')
        f.close()

    print("Finished")

except Exception as e:    
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 6\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")

In [ ]:
#Permanent cell 7

try:
    # create acronym HTML
    years = list(consolidation_events.simyear.unique())
    max_sim_year = max(years)
    max_first_year = max(list(consolidation_acronym.firstyear.unique()))

    html_path = map_folder + '\\Acronym_Summary'
    if not os.path.exists(html_path):
        os.makedirs(html_path)

    shutil.copy2('style.css', html_path + '\\style.css')
    shutil.copy2('script.js', html_path + '\\script.js')

    for consolidation_group in list(consolidation_acronym.consolidation_group.unique()):
        consolidation_group_df = consolidation_acronym[consolidation_acronym.consolidation_group == consolidation_group]

        f = open(html_path + '\\'+ consolidation_group + '.html', "w")
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write('<script src="script.js"></script>\n')
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write('<!DOCTYPE html>\n')
        f.write('<html>\n')
        f.write('<head>\n')
        f.write('<meta charset="utf-8">\n')
        f.write('</head>\n')
        f.write('<body>\n\n')

        f.write('<div class="tab">\n')
        for index, row in consolidation_group_df.iterrows():
            tab = row["Tab_ID"] + '(G)' if row["pmapprno"]==0 else row["Tab_ID"] + '(F)'
            color=row["consolidation_color"]
            color = color[3:].lower()
            bg_color = color_dict[color][0]
            text_color = color_dict[color][1]
            if color == "green": 
                color = "lightgreen"
            f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  
                    + ')" style="background-color:' + bg_color + '; color:' + text_color + ';">' + tab + '</button>\n')
        f.write('</div>\n')


        previous_tab = 'placeholder'
        for index, row in consolidation_group_df.iterrows():
            acronym = row["acronym"]
            tab = row["Tab_ID"] + '(G)' if row["pmapprno"]==0 else row["Tab_ID"] + '(F)'


            previous_tab = tab
        #     if ps == 'Katzie PS':
            if 1 == 1:

        #         if i == -1: #Make tab active ----This gives wrong zoom levels so it is temporarily disabled until fixed
        #             f.write('<div id="' + tab + '" class="tabcontent" style="display:block">\n')
        #         else:
                f.write('<div id="' + tab + '" class="tabcontent">\n')    

                header_text = "Gravity Main " + acronym if row["pmapprno"]==0 else "Force Main " + acronym
    #             f.write("<h1>" + header_text</h1>\n") 
                f.write("<h2>" + header_text + "</h2>\n") 

                exceedance_text = str(row['firstyear']) if max_first_year>row['firstyear'] else 'beyond ' + str(max_sim_year)
                exceedance_text = "First exceedance: " + exceedance_text
                f.write("<p>" + exceedance_text + "</p>\n") 

                pm_no = row["pmapprno"]

                acronym_pipes = consolidation_by_pipe.loc[(consolidation_by_pipe.consolidation_group == consolidation_group) &(consolidation_by_pipe.acronym == acronym) & (consolidation_by_pipe.pmapprno == pm_no)]
    #             details_events = consolidation_events.loc[(consolidation_events.Consolidation_Group == consolidation_group) & (consolidation_events.SimYear == simyear) & (consolidation_events.Max_V == max_v) & (consolidation_events.Max_WL_Above_SOH == max_WL_above_SOH)
    #                                                        & (consolidation_events.Color == color) & (consolidation_events.PmApprNo == pm_no)]

    #             f.write('<div class="row"><div class="column">\n')
                f.write('<div class="sidenav">\n')

                path = result_lookup_folder + r"\All_Longsections\Long_Section_" + acronym + ".html"

                f.write('<p><a href="' + path + '" target="_blank">Open Long Section</a></p>\n')

                       ################ Index 
                if pm_no == 0:
    #                

                    f.write('<table style=\'width: 40%;\'>\n')
                    f.write('<tr>\n')
                    f.write('<th>Legend</th>\n')
                    f.write('</tr>\n')
                    f.write('\n')
                    f.write('<tr><td style="background-color: black; color:white"> WL > GL and SOH </td></tr>\n')
                    f.write('<tr><td style="background-color: red; color:black"> WL < GL and > SOH </td></tr>\n')
    #                 f.write('<tr><td style="background-color: aqua; color:black"> WL +/- 5cm of SOH </td></tr>\n')
                    f.write('<tr><td style="background-color: blueviolet; color:white"> WL > GL and < SOH </td></tr>\n')
                    f.write('<tr><td style="background-color: gold; color:black"> WL > crown and < GL and < SOH </td></tr>\n')
                    f.write('<tr><td style="background-color: lightgreen; color:black"> WL < SOH and crown </td></tr>\n')

                    f.write('</table>\n')

                else:
                    f.write('<table>\n')
                    f.write('<tr>\n')
                    f.write('<th>Legend</th>\n')
                    f.write('</tr>\n')
                    f.write('\n')
                    f.write('<tr><td style="background-color: red; color:black"> velocity >= 3 m/s </td></tr>\n')          
                    f.write('<tr><td style="background-color: lightgreen; color:black"> velocity < 3 m/s </td></tr>\n')
                    f.write('</table>\n')

             ################ Index  
                f.write('<br>\n')

                f.write('<table style=\'width: 50%;\'>\n')
                f.write('<tr>\n')
                f.write('<th rowspan="2">Pipe</th>\n') 

                f.write('<th rowspan="2">Node MUID</th>\n') 
                f.write('<th rowspan="2">Node Name</th>\n') 

                f.write('<th rowspan="2">Year Exceeded</th>\n')
                f.write('<th>Invert</th>\n')
                f.write('<th>Crown</th>\n')
                f.write('<th>SOH</th>\n')
                f.write('<th>Ground</th>\n')
    #    

                for _ in range(2):
                    for sim_year in years:
                        f.write('<th>' + str(sim_year) + '</th>\n')

                f.write('</tr>\n')

                f.write('<tr>\n')
                f.write('<th colspan="4" style="text-align: center">Elevation (m)</th>\n')
                value_header = "Max Water Level (m)" if pm_no == 0 else "Max Velocity (m/s)"
                value_cols = len(years)
                f.write('<th colspan="' + str(value_cols) + '">' + str(value_header) + '</th>\n')
                f.write('<th colspan="4" style="text-align: center">Max Discharge (L/s)</th>\n')
                f.write('</tr>\n')



                pm_no = row["pmapprno"]

                for index,row in acronym_pipes.iterrows():
                    f.write('<tr>\n')

    #                 add hyperlink


                    pipe = row['msm_link_muid']

                    path = result_lookup_folder + '\\All_Elements\\' + model_area + '_Discharge_Link_' + pipe + '.html'

                    f.write('<td><a href="' + path + '" target="_blank">'+ pipe + '</a></td>\n')



                    color=row["consolidation_color"]
                    color = color[3:].lower()
                    bg_color = color_dict[color][0]
                    text_color = color_dict[color][1]

    #                 </a>

                    pipe_key = consolidation_group + "-" + pipe + "-" + str(years[0])

    #                  add hyperlink 

                    node = consolidation_events.loc[pipe_key,'msm_node_muid']           
                    path = result_lookup_folder + '\\All_Elements\\' + model_area + '_WaterLevel_Node_' + node + '.html'

                    f.write('<td><a href="' + path + '" target="_blank">'+ node + '</a></td>\n')


                    assetname = str(consolidation_events.loc[pipe_key,'assetname'])
                    assetname = "" if assetname == 'nan' else assetname
                    f.write('<td>'+ assetname + '</td>\n')

                    first_year = row['firstyear'] if max_first_year>row['firstyear'] else '> ' + str(max_sim_year)
                    f.write('<td style="background-color:'+ bg_color + '; color:' + text_color + ';">' + str(first_year) + '</td>\n')


                    f.write('<td>'+ str(round(consolidation_events.loc[pipe_key,'invertlevel'],2)) + '</td>\n')
                    f.write('<td>'+ str(round(consolidation_events.loc[pipe_key,'pipecrown'],2)) + '</td>\n')

                    soh = consolidation_events.loc[pipe_key,'criticallevel']
                    soh = round(soh,2) if not np.isnan(soh) else ''
                    f.write('<td>'+ str(soh) + '</td>\n')
                    f.write('<td>'+ str(round(consolidation_events.loc[pipe_key,'groundlevel'],2)) + '</td>\n')





                    for sim_year in years:
                        pipe_key = consolidation_group + "-" + pipe + "-" + str(sim_year)
                        try:
                            color = consolidation_events.loc[pipe_key,"color"].lower()
                            bg_color = color_dict[color][0]
                            text_color = color_dict[color][1]



                            if pm_no == 0:
                                value = consolidation_events.loc[pipe_key,"maxwl"]
                            else:
                                value = consolidation_events.loc[pipe_key,"maxv"]
                            if type(value) == np.nan:
                                value = ""
                            else:
                                value = round(value,2)


                            f.write('<td style="background-color:'+ bg_color + '; color:' + text_color + ';">' + str(value)+'</td>\n')
                        except:
                            pass

                    for sim_year in years:
                        pipe_key = consolidation_group + "-" + pipe + "-" + str(sim_year)
                        try:
                            discharge_value = consolidation_events.loc[pipe_key,"maxq"]*1000
                            f.write('<td>' + str(int(round(discharge_value, 0))) + '</td>\n')
                        except:
                            pass

                    f.write('</tr>\n')
                f.write('</table>\n')
                for i in range(10):
                    f.write('<h1 style="color: white">End of tables</h1>\n')#Invisible, just to enable scroll to table bottoms

    #             <a href="\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\HTML_Plots_V126\All_Longsections"acronym"" target="_blank">Open Long Section</a>


                f.write('</div>\n')


    #             f.write('<div class="column">\n')
                f.write('<div class="main">\n')


                if pm_no == 0:
                    map = acronym + "-GM-" + consolidation_group
                else:
                    map = acronym + "-FM-" + consolidation_group
                map_string = '<img src="Maps\\' + map + '.jpg" alt="' + map + '">\n'
                f.write(map_string + "\n")
                f.write('</div>\n')    
    #             f.write('</div>\n')

                f.write("</div>\n")  

        f.write('</body>\n')
        f.write('</html>\n')
        f.close()

    print("Finished")
    
except Exception as e:    
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 7\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")

